<a href="https://colab.research.google.com/github/ecohydro/DroughtEDM/blob/master/growing_season_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This repository contains functions for generating the pixelwise prediction skill of NDVI across East Africa 
    
    http://maps.elie.ucl.ac.be/CCI/viewer/download/ESACCI-LC-QuickUserGuide-LC-Maps_v2-0-7.pdf

In [6]:
from pyEDM import *
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import os
import geopandas as gpd
import rasterio
import xarray as xr
from rasterio.plot import show
from tqdm import tqdm
import datetime
from scipy import stats

In [7]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
prediction_skill_interval = '/home/rgreen/DroughtEDM/Data/Prediction_skill_interval/'
prediction_skill_ed = '/home/rgreen/DroughtEDM/Data/Prediction_skill_ed/'
prediction_skill_ed_anom_hh = '/home/rgreen/DroughtEDM/Data/Prediction_skill_ed/ndvi_anom_hh/'

EA_prediction_skill= '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/'
pred_skill_clim = '/home/rgreen/DroughtEDM/Data/Prediction_skill_climatology/'
anomaly_maps = '/home/rgreen/DroughtEDM/Data/Anomaly_Maps/'

In [10]:
et_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Hobbins_ET/ETos_p05_EA/'
ndvi_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/EA/'
sm10_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/EA10/'
sm40_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/EA40/'
sm100_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/EA100/'
sm200_ea = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/EA200/'

In [11]:
et_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/Hobbins_ET/Anomaly_Norm/'
ndvi_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/eMODIS_NDVI/Anomaly_Norm/'
sm10_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly10_Norm/'
sm40_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly40_Norm/'
sm100_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly100_Norm/'
sm200_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/FLDAS_SM/Anomaly200_Norm/'

In [12]:
# broadleaf_open_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_broadleaf_open/'
# crop_irr_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_crop_irr/'
# crop_rain_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_crop_rain/'
# grassland_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_grassland/'
# shrubland_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_shrubland/'

broadleaf_open_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_broadleaf_open_anom/'
crop_irr_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_crop_irr_anom/'
crop_rain_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_crop_rain_anom/'
grassland_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_grassland_anom/'
shrubland_anom_dir = '/home/rgreen/tana-spin/rgreen/DroughtEDM/Data/VariablesByLandCover16/variables_shrubland_anom/'

In [13]:
EA_prediction_skill = '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/'

In [14]:
# ea_adminbds = Admin_boundaries + 'gadm36_EastAfrica.shp'
# ea_bndry = gpd.read_file(ea_adminbds)
# world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# Africa_basemap = world[world['continent'] == 'Africa']
# EA_countries = ['Burundi', 'Djibouti', 'Eritrea', 'Ethiopia', 'Kenya', 'Rwanda', 'Sudan', 'S. Sudan', 'Somalia', 'Somaliland', 'South Sudan', 'Tanzania', 'Uganda']
# EastAfrica_basemap = world[world['name'].isin(EA_countries)]
# EastAfrica_basemap.to_file(Admin_boundaries + 'EastAfrica_countries.shp')

In [15]:
def generate_dekads(start, end):
    '''
    Generates list of timestamps of dekads between two dates
    
    Parameters
    ----------
    start : pd.datetime object
        Start date of list
    end : pd.datetime object
        end date of list
    '''
    
    dtrange = pd.date_range(start, end)
    days = list(range(len(dtrange))) #length of dtrange
    daysDF = pd.DataFrame({'Days': days}, index=dtrange)
    d = daysDF.index.day - np.clip((daysDF.index.day-1) // 10, 0, 2)*10 - 1 
    dekaddates = daysDF.index.values - np.array(d, dtype="timedelta64[D]")
    dekads = daysDF.groupby(dekaddates).mean()
    dekads = dekads.index
    
    return dekads

In [16]:
start = pd.datetime(2002,7,1)
end = pd.datetime(2019,4,30)

dekads = generate_dekads(start, end)

## 2. Pixelwise Prediction Skill Across East Africa

In [17]:
def pixelwise_ts_table(in_dir, keyword):
    
    '''
    This function searches through the file directory and creates a dataframe of pixel values over time from rasters
    based on a given keyword describing the environmental variable of interest to generate a raster time series stack 
    
   In the output dataframe, columns are each pixel extracted from the raster (read squentially into a list)
   and rows are the same pixel over time (dekads going down)
    
    Args:
    - in_dir: path to the input directory
    -keyword: string that is unique to the environmental variable
        options: ['precip', 'temp', 'ndvi']
    '''
    
    
    files=np.array(sorted(os.listdir(in_dir)))
    tifs = pd.Series(files).str.contains(keyword)
    files = files[tifs]
    #n_samples = len(files)
    
    
    pixelwise_TS = []
    
    for filename in tqdm(files): 
        
        
            open_file = xr.open_rasterio(in_dir+filename).sel(band=1)
            array = open_file.values
            #
            pixel_list = array.ravel().tolist()
            
            pixelwise_TS.append(pixel_list)

    return pd.DataFrame(pixelwise_TS)

    

In [28]:
et_table_anom = pixelwise_ts_table(et_anom_dir, 'et')
ndvi_table_anom = pixelwise_ts_table(ndvi_anom_dir, 'ndvi')
sm10_table_anom = pixelwise_ts_table(sm10_anom_dir, 'sm10_')
sm40_table_anom = pixelwise_ts_table(sm40_anom_dir, 'sm40')
sm100_table_anom = pixelwise_ts_table(sm100_anom_dir, 'sm100')
sm200_table_anom = pixelwise_ts_table(sm200_anom_dir, 'sm200')

100%|██████████| 606/606 [00:17<00:00, 33.91it/s]


In [ ]:
et_bo_table_anom = pixelwise_ts_table(broadleaf_open_anom_dir, 'et')
ndvi_bo_table_anom = pixelwise_ts_table(broadleaf_open_anom_dir, 'ndvi')
sm10_bo_table_anom = pixelwise_ts_table(broadleaf_open_anom_dir, 'sm10_')
sm40_bo_table_anom = pixelwise_ts_table(broadleaf_open_anom_dir, 'sm40')
sm100_bo_table_anom = pixelwise_ts_table(broadleaf_open_anom_dir, 'sm100')
sm200_bo_table_anom = pixelwise_ts_table(broadleaf_open_anom_dir, 'sm200')

In [14]:
et_ci_table_anom = pixelwise_ts_table(crop_irr_anom_dir, 'et')
ndvi_ci_table_anom = pixelwise_ts_table(crop_irr_anom_dir,  'ndvi')
sm10_ci_table_anom = pixelwise_ts_table(crop_irr_anom_dir, 'sm10_')
sm40_ci_table_anom = pixelwise_ts_table(crop_irr_anom_dir, 'sm40')
sm100_ci_table_anom = pixelwise_ts_table(crop_irr_anom_dir, 'sm100')
sm200_ci_table_anom = pixelwise_ts_table(crop_irr_anom_dir, 'sm200')

100%|██████████| 606/606 [00:08<00:00, 68.09it/s]


In [15]:
et_cr_table_anom = pixelwise_ts_table(crop_rain_anom_dir, 'et')
ndvi_cr_table_anom = pixelwise_ts_table(crop_rain_anom_dir,  'ndvi')
sm10_cr_table_anom = pixelwise_ts_table(crop_rain_anom_dir, 'sm10_')
sm40_cr_table_anom = pixelwise_ts_table(crop_rain_anom_dir, 'sm40')
sm100_cr_table_anom = pixelwise_ts_table(crop_rain_anom_dir, 'sm100')
sm200_cr_table_anom = pixelwise_ts_table(crop_rain_anom_dir, 'sm200')

100%|██████████| 606/606 [00:08<00:00, 72.52it/s]


In [16]:
et_gr_table_anom = pixelwise_ts_table(grassland_anom_dir, 'et')
ndvi_gr_table_anom = pixelwise_ts_table(grassland_anom_dir,  'ndvi')
sm10_gr_table_anom = pixelwise_ts_table(grassland_anom_dir, 'sm10_')
sm40_gr_table_anom = pixelwise_ts_table(grassland_anom_dir, 'sm40')
sm100_gr_table_anom = pixelwise_ts_table(grassland_anom_dir, 'sm100')
sm200_gr_table_anom = pixelwise_ts_table(grassland_anom_dir, 'sm200')

100%|██████████| 606/606 [00:08<00:00, 71.23it/s]


In [17]:
et_sh_table_anom = pixelwise_ts_table(shrubland_anom_dir, 'et')
ndvi_sh_table_anom = pixelwise_ts_table(shrubland_anom_dir,  'ndvi')
sm10_sh_table_anom = pixelwise_ts_table(shrubland_anom_dir, 'sm10_')
sm40_sh_table_anom = pixelwise_ts_table(shrubland_anom_dir, 'sm40')
sm100_sh_table_anom = pixelwise_ts_table(shrubland_anom_dir, 'sm100')
sm200_sh_table_anom = pixelwise_ts_table(shrubland_anom_dir, 'sm200')

100%|██████████| 606/606 [00:08<00:00, 75.73it/s]


In [29]:
et_table_anom.label = 'et'
et_table_anom.rs_rows = 674
et_table_anom.rs_cols = 583
et_table_anom.n_samples = 606
ndvi_table_anom.label = 'ndvi'
ndvi_table_anom.rs_rows = 674
ndvi_table_anom.rs_cols = 583
ndvi_table_anom.n_samples = 606
sm10_table_anom.label = 'sm'
sm10_table_anom.rs_rows = 674
sm10_table_anom.rs_cols = 583
sm10_table_anom.n_samples = 606
sm40_table_anom.label = 'sm'
sm40_table_anom.rs_rows = 674
sm40_table_anom.rs_cols = 583
sm40_table_anom.n_samples = 606
sm100_table_anom.label = 'sm'
sm100_table_anom.rs_rows = 674
sm100_table_anom.rs_cols = 583
sm100_table_anom.n_samples = 606
sm200_table_anom.label = 'sm'
sm200_table_anom.rs_rows = 674
sm200_table_anom.rs_cols = 583
sm200_table_anom.n_samples = 606

table_list_anom_sm10et = [et_table_anom, ndvi_table_anom, sm10_table_anom]
table_list_anom_sm40et = [et_table_anom, ndvi_table_anom, sm40_table_anom]
table_list_anom_sm100et = [et_table_anom, ndvi_table_anom, sm100_table_anom]
table_list_anom_sm200et = [et_table_anom, ndvi_table_anom, sm200_table_anom]

In [20]:
et_bo_table_anom.label = 'et'
et_bo_table_anom.rs_rows = 674
et_bo_table_anom.rs_cols = 583
et_bo_table_anom.n_samples = 606
ndvi_bo_table_anom.label = 'ndvi'
ndvi_bo_table_anom.rs_rows = 674
ndvi_bo_table_anom.rs_cols = 583
ndvi_bo_table_anom.n_samples = 606
sm10_bo_table_anom.label = 'sm'
sm10_bo_table_anom.rs_rows = 674
sm10_bo_table_anom.rs_cols = 583
sm10_bo_table_anom.n_samples = 606
sm40_bo_table_anom.label = 'sm'
sm40_bo_table_anom.rs_rows = 674
sm40_bo_table_anom.rs_cols = 583
sm40_bo_table_anom.n_samples = 606
sm100_bo_table_anom.label = 'sm'
sm100_bo_table_anom.rs_rows = 674
sm100_bo_table_anom.rs_cols = 583
sm100_bo_table_anom.n_samples = 606
sm200_bo_table_anom.label = 'sm'
sm200_bo_table_anom.rs_rows = 674
sm200_bo_table_anom.rs_cols = 583
sm200_bo_table_anom.n_samples = 606

table_list_bo_anom_sm10et = [et_bo_table_anom, ndvi_bo_table_anom, sm10_bo_table_anom]
table_list_bo_anom_sm40et = [et_bo_table_anom, ndvi_bo_table_anom, sm40_bo_table_anom]
table_list_bo_anom_sm100et = [et_bo_table_anom, ndvi_bo_table_anom, sm100_bo_table_anom]
table_list_bo_anom_sm200et = [et_bo_table_anom, ndvi_bo_table_anom, sm200_bo_table_anom]

In [50]:
et_ci_table_anom.label = 'et'
et_ci_table_anom.rs_rows = 674
et_ci_table_anom.rs_cols = 583
et_ci_table_anom.n_samples = 606
ndvi_ci_table_anom.label = 'ndvi'
ndvi_ci_table_anom.rs_rows = 674
ndvi_ci_table_anom.rs_cols = 583
sm10_ci_table_anom.label = 'sm'
sm10_ci_table_anom.rs_rows = 674
sm10_ci_table_anom.rs_cols = 583
sm10_ci_table_anom.n_samples = 606
sm40_ci_table_anom.label = 'sm'
sm40_ci_table_anom.rs_rows = 674
sm40_ci_table_anom.rs_cols = 583
sm40_ci_table_anom.n_samples = 606
sm100_ci_table_anom.label = 'sm'
sm100_ci_table_anom.rs_rows = 674
sm100_ci_table_anom.rs_cols = 583
sm100_ci_table_anom.n_samples = 606
sm200_ci_table_anom.label = 'sm'
sm200_ci_table_anom.rs_rows = 674
sm200_ci_table_anom.rs_cols = 583
sm200_ci_table_anom.n_samples = 606

table_list_ci_anom_sm10et = [et_ci_table_anom, ndvi_ci_table_anom, sm10_ci_table_anom]
table_list_ci_anom_sm40et = [et_ci_table_anom, ndvi_ci_table_anom, sm40_ci_table_anom]
table_list_ci_anom_sm100et = [et_ci_table_anom, ndvi_ci_table_anom, sm100_ci_table_anom]
table_list_ci_anom_sm200et = [et_ci_table_anom, ndvi_ci_table_anom, sm200_ci_table_anom]

In [20]:
et_cr_table_anom.label = 'et'
et_cr_table_anom.rs_rows = 674
et_cr_table_anom.rs_cols = 583
et_cr_table_anom.n_samples = 606
ndvi_cr_table_anom.label = 'ndvi'
ndvi_cr_table_anom.rs_rows = 674
ndvi_cr_table_anom.rs_cols = 583
ndvi_cr_table_anom.n_samples = 606
sm10_cr_table_anom.label = 'sm'
sm10_cr_table_anom.rs_rows = 674
sm10_cr_table_anom.rs_cols = 583
sm10_cr_table_anom.n_samples = 606
sm40_cr_table_anom.label = 'sm'
sm40_cr_table_anom.rs_rows = 674
sm40_cr_table_anom.rs_cols = 583
sm40_cr_table_anom.n_samples = 606
sm100_cr_table_anom.label = 'sm'
sm100_cr_table_anom.rs_rows = 674
sm100_cr_table_anom.rs_cols = 583
sm100_cr_table_anom.n_samples = 606
sm200_cr_table_anom.label = 'sm'
sm200_cr_table_anom.rs_rows = 674
sm200_cr_table_anom.rs_cols = 583
sm200_cr_table_anom.n_samples = 606

table_list_cr_anom_sm10et = [et_cr_table_anom, ndvi_cr_table_anom, sm10_cr_table_anom]
table_list_cr_anom_sm40et = [et_cr_table_anom, ndvi_cr_table_anom, sm40_cr_table_anom]
table_list_cr_anom_sm100et = [et_cr_table_anom, ndvi_cr_table_anom, sm100_cr_table_anom]
table_list_cr_anom_sm200et = [et_cr_table_anom, ndvi_cr_table_anom, sm200_cr_table_anom]

In [45]:
et_gr_table_anom.label = 'et'
et_gr_table_anom.rs_rows = 674
et_gr_table_anom.rs_cols = 583
et_gr_table_anom.n_samples = 606
ndvi_gr_table_anom.label = 'ndvi'
ndvi_gr_table_anom.rs_rows = 674
ndvi_gr_table_anom.rs_cols = 583
ndvi_gr_table_anom.n_samples = 606
sm10_gr_table_anom.label = 'sm'
sm10_gr_table_anom.rs_rows = 674
sm10_gr_table_anom.rs_cols = 583
sm10_gr_table_anom.n_samples = 606
sm40_gr_table_anom.label = 'sm'
sm40_gr_table_anom.rs_rows = 674
sm40_gr_table_anom.rs_cols = 583
sm40_gr_table_anom.n_samples = 606
sm100_gr_table_anom.label = 'sm'
sm100_gr_table_anom.rs_rows = 674
sm100_gr_table_anom.rs_cols = 583
sm100_gr_table_anom.n_samples = 606
sm200_gr_table_anom.label = 'sm'
sm200_gr_table_anom.rs_rows = 674
sm200_gr_table_anom.rs_cols = 583
sm200_gr_table_anom.n_samples = 606

table_list_gr_anom_sm10et = [et_gr_table_anom, ndvi_gr_table_anom, sm10_gr_table_anom]
table_list_gr_anom_sm40et = [et_gr_table_anom, ndvi_gr_table_anom, sm40_gr_table_anom]
table_list_gr_anom_sm100et = [et_gr_table_anom, ndvi_gr_table_anom, sm100_gr_table_anom]
table_list_gr_anom_sm200et = [et_gr_table_anom, ndvi_gr_table_anom, sm200_gr_table_anom]

In [46]:
et_sh_table_anom.label = 'et'
et_sh_table_anom.rs_rows = 674
et_sh_table_anom.rs_cols = 583
et_sh_table_anom.n_samples = 606
ndvi_sh_table_anom.label = 'ndvi'
ndvi_sh_table_anom.rs_rows = 674
ndvi_sh_table_anom.rs_cols = 583
ndvi_sh_table_anom.n_samples = 606
sm10_sh_table_anom.label = 'sm'
sm10_sh_table_anom.rs_rows = 674
sm10_sh_table_anom.rs_cols = 583
sm10_sh_table_anom.n_samples = 606
sm40_sh_table_anom.label = 'sm'
sm40_sh_table_anom.rs_rows = 674
sm40_sh_table_anom.rs_cols = 583
sm40_sh_table_anom.n_samples = 606
sm100_sh_table_anom.label = 'sm'
sm100_sh_table_anom.rs_rows = 674
sm100_sh_table_anom.rs_cols = 583
sm100_sh_table_anom.n_samples = 606
sm200_sh_table_anom.label = 'sm'
sm200_sh_table_anom.rs_rows = 674
sm200_sh_table_anom.rs_cols = 583
sm200_sh_table_anom.n_samples = 606

table_list_sh_anom_sm10et = [et_sh_table_anom, ndvi_sh_table_anom, sm10_sh_table_anom]
table_list_sh_anom_sm40et = [et_sh_table_anom, ndvi_sh_table_anom, sm40_sh_table_anom]
table_list_sh_anom_sm100et = [et_sh_table_anom, ndvi_sh_table_anom, sm100_sh_table_anom]
table_list_sh_anom_sm200et = [et_sh_table_anom, ndvi_sh_table_anom, sm200_sh_table_anom]

In [3]:
def multi_pixelwise_simplex(tables, target_label, train, test, ed, pi, skill_score): #year month_list, 
    
    '''
    This function computes the simplex prediction skill for a given embedding dimensions, prediction interval, land cover and environmental variable
    across pixels given multiple tables of different variables of pixel values across time for a region from the function pixelwise_ts_table 

    The result is an np.array of simplex error values reshaped back into the shape of the original raster image
    
    Args:
    - table: list of input tables, where every table has columns with the time series of each pixel in a raster
    - target_label: text label for the table to be used as the target (e.g. 'ndvi')
    - train: portion of data library used to train (string of range with space between values)
    - test: portion of data libary used to test (string of range with space between values) 
    - ed: embedding dimension
    - pi: prediction interval (Tp) 
    - rs_rows: number of rows to reshape list into original raster size
    - rs_cols: number of columns to reshape list into original raster size
   
    '''

    table_dict = {'Time': dekads }

    simplex_out = []
    labels = ' '.join([table.label for table in tables])# Assumes that all tables have labels

    # Generate future dekads
    dekads_fut = generate_dekads(dekads[-1],pd.datetime(2020,4,30))
    
    # Iterate through columns (pixels)
    for col in tqdm(range(len(tables[0].columns))):
        # PIXEL DATAFRAME
        # Generate dataframe for pixel where columns correspond to variables and rows to timestamp (image)
        [table_dict.update({table.label: table.iloc[:,col]}) for i, table in enumerate(tables)]
        #interim_df = pd.DataFrame(table_dict, index = dekads)
        interim_df = pd.DataFrame(table_dict)

        interim_df.set_index('Time',inplace=True, drop=False)
        

    
        # If column has lots of nans (water), append nan
        if max([len(interim_df[interim_df[col].isnull()]) for col in interim_df.columns]) > 20:
            simplex_out.append(np.nan)
        
        else:

            # Check whether column has a few nans
            if 'et' in interim_df.columns and interim_df.et.isnull().values.any():
                # If so, iterate through nan rows and replace with monthly mean
                for t in interim_df[interim_df.et.isnull()].index:
                    interim_df.et.loc[t] = interim_df.et[interim_df.index.month == t.month].mean()
                    #print(interim_df)
            if interim_df.ndvi.isnull().values.any():
            # If so, iterate through nan rows and replace with monthly mean
                for t in interim_df[interim_df.ndvi.isnull()].index:
                    interim_df.ndvi.loc[t] = interim_df.ndvi[interim_df.index.month == t.month].mean()
            if 'sm' in interim_df.columns and interim_df.sm.isnull().values.any():
            # If so, iterate through nan rows and replace with monthly mean
                for t in interim_df[interim_df.sm.isnull()].index:
                    interim_df.sm.loc[t] = interim_df.sm[interim_df.index.month == t.month].mean()

            # Run simplex
            simplex_result = Simplex(dataFrame = interim_df,
                                     lib = train,
                                     pred = test,
                                     E = ed, Tp = pi,
                                     columns = labels, target = target_label, showPlot = False)

            
            # Add datetime index + fix dekad values for future values
            simplex_result.Time = pd.to_datetime(simplex_result.Time)
            simplex_result.set_index(simplex_result.Time, inplace=True)
            
            
            # Fix datetime index for future dekads
            simplex_result[-pi:].Time = dekads_fut[1:pi+1]
            simplex_result = simplex_result.dropna()
           # print(simplex_result)

            observations = np.array(simplex_result.Observations)
            predictions = np.array(simplex_result.Predictions)
            error = ComputeError(observations, predictions)
            
            if skill_score == 'mae':
                mae = list(error.values())[0]
                simplex_out.append(mae)
                
            if skill_score == 'rmse':
                rmse = list(error.values())[1]
                simplex_out.append(rmse)
                
            if skill_score == 'rho':
                rho = list(error.values())[2]
                simplex_out.append(rho)
                
            if skill_score =='nse':
                pdims = predictions.shape
                odims = observations.shape
                if not np.array_equal(pdims, odims):
                    message = 'predicted and reference field dimensions do not' + \
                        ' match.\n' + \
                        'shape(predicted)= ' + str(pdims) + ', ' + \
                        'shape(reference)= ' + str(odims) + \
                        '\npredicted type: ' + str(type(predictions))
                    raise ValueError(message)

                # Calculate the NSE
                nse = 1 - (np.sum((predictions - observations)**2) /
                           np.sum((observations - np.mean(predictions))**2))
                simplex_out.append(nse)
                #print(nse)

    return np.reshape(simplex_out, (tables[0].rs_rows, tables[0].rs_cols))

In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm10et_rho = multi_pixelwise_simplex(table_list_bo_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm10et_rho = multi_pixelwise_simplex(table_list_sh_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm10et_rho = multi_pixelwise_simplex(table_list_gr_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm10et_rho = multi_pixelwise_simplex(table_list_ci_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm10et_rho = multi_pixelwise_simplex(table_list_cr_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


 55%|█████▍    | 214628/392942 [15:41<09:42, 306.15it/s] 

In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm40et_rho = multi_pixelwise_simplex(table_list_bo_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm40et_rho = multi_pixelwise_simplex(table_list_sh_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


In [52]:
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm40et_rho = multi_pixelwise_simplex(table_list_gr_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm40et_rho = multi_pixelwise_simplex(table_list_ci_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm40et_rho = multi_pixelwise_simplex(table_list_cr_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


100%|██████████| 392942/392942 [24:31<00:00, 267.09it/s] 


In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm100et_rho = multi_pixelwise_simplex(table_list_bo_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm100et_rho = multi_pixelwise_simplex(table_list_sh_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm100et_rho = multi_pixelwise_simplex(table_list_gr_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm100et_rho = multi_pixelwise_simplex(table_list_ci_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm100et_rho = multi_pixelwise_simplex(table_list_cr_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


 47%|████▋     | 184844/392942 [12:31<20:48, 166.73it/s] 

In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm200et_rho = multi_pixelwise_simplex(table_list_bo_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm200et_rho = multi_pixelwise_simplex(table_list_sh_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm200et_rho = multi_pixelwise_simplex(table_list_gr_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm200et_rho = multi_pixelwise_simplex(table_list_ci_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm200et_rho = multi_pixelwise_simplex(table_list_cr_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm10et_nse = multi_pixelwise_simplex(table_list_bo_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm10et_nse = multi_pixelwise_simplex(table_list_sh_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm10et_nse = multi_pixelwise_simplex(table_list_gr_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm10et_nse = multi_pixelwise_simplex(table_list_ci_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm10et_nse = multi_pixelwise_simplex(table_list_cr_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')


In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm40et_nse = multi_pixelwise_simplex(table_list_bo_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm40et_nse = multi_pixelwise_simplex(table_list_sh_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm40et_nse = multi_pixelwise_simplex(table_list_gr_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm40et_nse = multi_pixelwise_simplex(table_list_ci_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm40et_nse = multi_pixelwise_simplex(table_list_cr_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')


In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm100et_nse = multi_pixelwise_simplex(table_list_bo_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm100et_nse = multi_pixelwise_simplex(table_list_sh_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm100et_nse = multi_pixelwise_simplex(table_list_gr_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm100et_nse = multi_pixelwise_simplex(table_list_ci_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm100et_nse = multi_pixelwise_simplex(table_list_cr_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')


In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm200et_nse = multi_pixelwise_simplex(table_list_bo_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm200et_nse = multi_pixelwise_simplex(table_list_sh_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm200et_nse = multi_pixelwise_simplex(table_list_gr_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm200et_nse = multi_pixelwise_simplex(table_list_ci_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm200et_nse = multi_pixelwise_simplex(table_list_cr_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')


In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_s10met_mae = multi_pixelwise_simplex(table_list_bo_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_s10met_mae = multi_pixelwise_simplex(table_list_sh_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_s10met_mae = multi_pixelwise_simplex(table_list_gr_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_s10met_mae = multi_pixelwise_simplex(table_list_ci_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_s10met_mae = multi_pixelwise_simplex(table_list_cr_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')


In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_s40met_mae = multi_pixelwise_simplex(table_list_bo_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_s40met_mae = multi_pixelwise_simplex(table_list_sh_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_s40met_mae = multi_pixelwise_simplex(table_list_gr_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_s40met_mae = multi_pixelwise_simplex(table_list_ci_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_s40met_mae = multi_pixelwise_simplex(table_list_cr_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')


In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_s100met_mae = multi_pixelwise_simplex(table_list_bo_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_s100met_mae = multi_pixelwise_simplex(table_list_sh_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_s100met_mae = multi_pixelwise_simplex(table_list_gr_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_s100met_mae = multi_pixelwise_simplex(table_list_ci_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_s100met_mae = multi_pixelwise_simplex(table_list_cr_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')


In [ ]:
ndvi_ea_simplex_multi_bo_pi18_ed6_anom_s200met_mae = multi_pixelwise_simplex(table_list_bo_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_sh_pi18_ed6_anom_s200met_mae = multi_pixelwise_simplex(table_list_sh_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_gr_pi18_ed6_anom_s200met_mae = multi_pixelwise_simplex(table_list_gr_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_ci_pi18_ed6_anom_s200met_mae = multi_pixelwise_simplex(table_list_ci_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_multi_cr_pi18_ed6_anom_s200met_mae = multi_pixelwise_simplex(table_list_cr_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_et_rho = multi_pixelwise_simplex([et_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_sh_pi18_ed6_anom_et_rho = multi_pixelwise_simplex([et_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_gr_pi18_ed6_anom_et_rho = multi_pixelwise_simplex([et_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_ci_pi18_ed6_anom_et_rho = multi_pixelwise_simplex([et_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_cr_pi18_ed6_anom_et_rho = multi_pixelwise_simplex([et_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


 35%|███▌      | 138943/392942 [05:50<17:04, 247.85it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 55%|█████▍    | 214341/392942 [17:30<29:36, 100.56it/s] 

In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm10_rho = multi_pixelwise_simplex([sm10_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm10_rho = multi_pixelwise_simplex([sm10_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm10_rho = multi_pixelwise_simplex([sm10_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm10_rho = multi_pixelwise_simplex([sm10_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm10_rho = multi_pixelwise_simplex([sm10_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm40_rho = multi_pixelwise_simplex([sm40_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm40_rho = multi_pixelwise_simplex([sm40_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm40_rho = multi_pixelwise_simplex([sm40_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm40_rho = multi_pixelwise_simplex([sm40_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm40_rho = multi_pixelwise_simplex([sm40_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm100_rho = multi_pixelwise_simplex([sm100_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm100_rho = multi_pixelwise_simplex([sm100_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm100_rho = multi_pixelwise_simplex([sm100_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm100_rho = multi_pixelwise_simplex([sm100_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm100_rho = multi_pixelwise_simplex([sm100_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm200_rho = multi_pixelwise_simplex([sm200_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm200_rho = multi_pixelwise_simplex([sm200_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm200_rho = multi_pixelwise_simplex([sm200_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm200_rho = multi_pixelwise_simplex([sm200_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm200_rho = multi_pixelwise_simplex([sm200_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_et_nse = multi_pixelwise_simplex([et_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_sh_pi18_ed6_anom_et_nse = multi_pixelwise_simplex([et_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_gr_pi18_ed6_anom_et_nse = multi_pixelwise_simplex([et_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_ci_pi18_ed6_anom_et_nse = multi_pixelwise_simplex([et_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_cr_pi18_ed6_anom_et_nse = multi_pixelwise_simplex([et_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')


  4%|▍         | 15998/392942 [00:34<13:16, 473.12it/s]

In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm10_nse = multi_pixelwise_simplex([sm10_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm10_nse = multi_pixelwise_simplex([sm10_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm10_nse = multi_pixelwise_simplex([sm10_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm10_nse = multi_pixelwise_simplex([sm10_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm10_nse = multi_pixelwise_simplex([sm10_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm40_nse = multi_pixelwise_simplex([sm40_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm40_nse = multi_pixelwise_simplex([sm40_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm40_nse = multi_pixelwise_simplex([sm40_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm40_nse = multi_pixelwise_simplex([sm40_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm40_nse = multi_pixelwise_simplex([sm40_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm100_nse = multi_pixelwise_simplex([sm100_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm100_nse = multi_pixelwise_simplex([sm100_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm100_nse = multi_pixelwise_simplex([sm100_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm100_nse = multi_pixelwise_simplex([sm100_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm100_nse = multi_pixelwise_simplex([sm100_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm200_nse = multi_pixelwise_simplex([sm200_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm200_nse = multi_pixelwise_simplex([sm200_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm200_nse = multi_pixelwise_simplex([sm200_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm200_nse = multi_pixelwise_simplex([sm200_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm200_nse = multi_pixelwise_simplex([sm200_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_et_mae = multi_pixelwise_simplex([et_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_sh_pi18_ed6_anom_et_mae = multi_pixelwise_simplex([et_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_gr_pi18_ed6_anom_et_mae = multi_pixelwise_simplex([et_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_ci_pi18_ed6_anom_et_mae = multi_pixelwise_simplex([et_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_cr_pi18_ed6_anom_et_mae = multi_pixelwise_simplex([et_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm10_mae = multi_pixelwise_simplex([sm10_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm10_mae = multi_pixelwise_simplex([sm10_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm10_mae = multi_pixelwise_simplex([sm10_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm10_mae = multi_pixelwise_simplex([sm10_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm10_mae = multi_pixelwise_simplex([sm10_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm40_mae = multi_pixelwise_simplex([sm40_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm40_mae = multi_pixelwise_simplex([sm40_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm40_mae = multi_pixelwise_simplex([sm40_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm40_mae = multi_pixelwise_simplex([sm40_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm40_mae = multi_pixelwise_simplex([sm40_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm100_mae = multi_pixelwise_simplex([sm100_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm100_mae = multi_pixelwise_simplex([sm100_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm100_mae = multi_pixelwise_simplex([sm100_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm100_mae = multi_pixelwise_simplex([sm100_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm100_mae = multi_pixelwise_simplex([sm100_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')


In [ ]:
ndvi_ea_simplex_bo_pi18_ed6_anom_sm200_mae = multi_pixelwise_simplex([sm200_bo_table_anom, ndvi_bo_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_sh_pi18_ed6_anom_sm200_mae = multi_pixelwise_simplex([sm200_sh_table_anom, ndvi_sh_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_gr_pi18_ed6_anom_sm200_mae = multi_pixelwise_simplex([sm200_gr_table_anom, ndvi_gr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_ci_pi18_ed6_anom_sm200_mae = multi_pixelwise_simplex([sm200_ci_table_anom, ndvi_ci_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_cr_pi18_ed6_anom_sm200_mae = multi_pixelwise_simplex([sm200_cr_table_anom, ndvi_cr_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')


In [ ]:
ndvi_ea_simplex_pi18_ed6_anom_sm10et_rho = multi_pixelwise_simplex(table_list_anom_sm10et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_pi18_ed6_anom_sm40et_rho = multi_pixelwise_simplex(table_list_anom_sm40et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_pi18_ed6_anom_sm100et_rho = multi_pixelwise_simplex(table_list_anom_sm100et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_pi18_ed6_anom_sm200et_rho = multi_pixelwise_simplex(table_list_anom_sm200et, 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


 12%|█▏        | 46034/392942 [15:05<2:02:44, 47.11it/s]

In [ ]:
ndvi_ea_simplex_pi18_ed6_anom_sm10_rho = multi_pixelwise_simplex([ndvi_table_anom, sm10_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_pi18_ed6_anom_sm40_rho = multi_pixelwise_simplex([ndvi_table_anom, sm40_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_pi18_ed6_anom_sm100_rho = multi_pixelwise_simplex([ndvi_table_anom, sm100_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')
ndvi_ea_simplex_pi18_ed6_anom_sm200_rho = multi_pixelwise_simplex([ndvi_table_anom, sm200_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')

ndvi_ea_simplex_pi18_ed6_anom_et_rho = multi_pixelwise_simplex([ndvi_table_anom, et_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'rho')


 12%|█▏        | 47281/392942 [14:14<1:55:19, 49.95it/s]

In [ ]:
ndvi_ea_simplex_pi18_ed6_anom_sm10_mae = multi_pixelwise_simplex([ndvi_table_anom, sm10_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_pi18_ed6_anom_sm40_mae = multi_pixelwise_simplex([ndvi_table_anom, sm40_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_pi18_ed6_anom_sm100_mae = multi_pixelwise_simplex([ndvi_table_anom, sm100_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')
ndvi_ea_simplex_pi18_ed6_anom_sm200_mae = multi_pixelwise_simplex([ndvi_table_anom, sm200_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')

ndvi_ea_simplex_pi18_ed6_anom_et_mae = multi_pixelwise_simplex([ndvi_table_anom, et_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'mae')


In [ ]:
ndvi_ea_simplex_pi18_ed6_anom_sm10_nse = multi_pixelwise_simplex([ndvi_table_anom, sm10_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_pi18_ed6_anom_sm40_nse = multi_pixelwise_simplex([ndvi_table_anom, sm40_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_pi18_ed6_anom_sm100_nse = multi_pixelwise_simplex([ndvi_table_anom, sm100_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')
ndvi_ea_simplex_pi18_ed6_anom_sm200_nse = multi_pixelwise_simplex([ndvi_table_anom, sm200_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')

ndvi_ea_simplex_pi18_ed6_anom_et_nse = multi_pixelwise_simplex([ndvi_table_anom, et_table_anom], 'ndvi', train = '1 303', test = '304 606', ed = 6, pi = 18, skill_score = 'nse')


# Save out rasters

In [25]:
def array2tiff(array, in_dir, out_dir, out_name):
    
    '''
    This function converts a 2D array into a tif using the profile of an existing tif

    '''
    

    with rasterio.open(in_dir+ os.listdir(in_dir)[0]) as src:
        profile = src.profile

    with rasterio.open(out_dir + out_name + '.tif', 'w', **profile) as dst:
        data = array
        data = data.astype(np.float32)
        #data[data<=0] = np.nan
        dst.write(data, indexes=1)

In [34]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_smet, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_smet')

In [35]:
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_smet, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_smet')

In [38]:
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_smet, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_smet')

In [39]:
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_smet, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_smet')

In [40]:
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_smet, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_smet')

In [77]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_smet_nse, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_smet_nse')

In [80]:
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_smet_nse, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_smet_nse')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_smet_nse, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_smet_nse')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_smet_nse, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_smet_nse')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_smet_nse, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_smet_nse')


In [83]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_smet_mae, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_smet_mae')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_smet_mae, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_smet_mae')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_smet_mae, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_smet_mae')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_smet_mae, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_smet_mae')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_smet_mae, ndvi_ea, '/home/rgreen/DroughtEDM/Data/EA_prediction_skill/', 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_smet_mae')


In [69]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm10et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm10et_rho')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm10et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm10et_rho')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm10et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm10et_rho')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm10et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm10et_rho')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm10et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm10et_rho')

In [72]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm40et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm40et_rho')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm40et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm40et_rho')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm40et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm40et_rho')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm40et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm40et_rho')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm40et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm40et_rho')

In [73]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm100et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm100et_rho')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm100et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm100et_rho')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm100et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm100et_rho')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm100et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm100et_rho')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm100et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm100et_rho')

In [74]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm200et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm200et_rho')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm200et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm200et_rho')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm200et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm200et_rho')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm200et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm200et_rho')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm200et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm200et_rho')

In [75]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm10et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm10et_nse')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm10et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm10et_nse')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm10et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm10et_nse')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm10et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm10et_nse')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm10et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm10et_nse')

In [76]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm40et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm40et_nse')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm40et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm40et_nse')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm40et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm40et_nse')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm40et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm40et_nse')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm40et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm40et_nse')

In [77]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm100et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm100et_nse')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm100et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm100et_nse')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm100et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm100et_nse')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm100et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm100et_nse')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm100et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm100et_nse')

In [78]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm200et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm200et_nse')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm200et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm200et_nse')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm200et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm200et_nse')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm200et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm200et_nse')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm200et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm200et_nse')

In [82]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_s10met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm10et_mae')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_s10met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm10et_mae')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_s10met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm10et_mae')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_s10met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm10et_mae')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_s10met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm10et_mae')

In [83]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_s40met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm40et_mae')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_s40met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm40et_mae')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_s40met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm40et_mae')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_s40met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm40et_mae')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_s40met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm40et_mae')

In [84]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_s100met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm100et_mae')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_s100met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm100et_mae')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_s100met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm100et_mae')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_s100met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm100et_mae')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_s100met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm100et_mae')

In [85]:
array2tiff(ndvi_ea_simplex_multi_bo_pi18_ed6_anom_s200met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_bo_pi18_ed6_anom_sm200et_mae')
array2tiff(ndvi_ea_simplex_multi_sh_pi18_ed6_anom_s200met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_sh_pi18_ed6_anom_sm200et_mae')
array2tiff(ndvi_ea_simplex_multi_gr_pi18_ed6_anom_s200met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_gr_pi18_ed6_anom_sm200et_mae')
array2tiff(ndvi_ea_simplex_multi_ci_pi18_ed6_anom_s200met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_ci_pi18_ed6_anom_sm200et_mae')
array2tiff(ndvi_ea_simplex_multi_cr_pi18_ed6_anom_s200met_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_multi_cr_pi18_ed6_anom_sm200et_mae')

In [103]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_et_rho')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_et_rho')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_et_rho')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_et_rho')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_et_rho')

In [104]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm10_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm10_rho')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm10_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm10_rho')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm10_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm10_rho')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm10_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm10_rho')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm10_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm10_rho')

In [105]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm40_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm40_rho')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm40_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm40_rho')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm40_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm40_rho')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm40_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm40_rho')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm40_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm40_rho')

In [106]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm100_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm100_rho')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm100_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm100_rho')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm100_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm100_rho')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm100_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm100_rho')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm100_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm100_rho')

In [107]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm200_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm200_rho')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm200_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm200_rho')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm200_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm200_rho')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm200_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm200_rho')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm200_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm200_rho')

In [120]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_et_nse')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_et_nse')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_et_nse')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_et_nse')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_et_nse')

In [121]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm10_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm10_nse')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm10_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm10_nse')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm10_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm10_nse')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm10_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm10_nse')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm10_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm10_nse')

In [122]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm40_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm40_nse')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm40_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm40_nse')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm40_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm40_nse')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm40_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm40_nse')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm40_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm40_nse')

In [123]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm100_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm100_nse')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm100_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm100_nse')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm100_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm100_nse')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm100_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm100_nse')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm100_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm100_nse')

In [124]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm200_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm200_nse')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm200_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm200_nse')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm200_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm200_nse')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm200_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm200_nse')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm200_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm200_nse')

In [125]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_et_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_et_mae')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_et_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_et_mae')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_et_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_et_mae')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_et_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_et_mae')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_et_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_et_mae')

In [126]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm10_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm10_mae')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm10_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm10_mae')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm10_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm10_mae')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm10_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm10_mae')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm10_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm10_mae')

In [127]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm40_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm40_mae')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm40_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm40_mae')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm40_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm40_mae')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm40_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm40_mae')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm40_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm40_mae')

In [128]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm100_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm100_mae')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm100_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm100_mae')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm100_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm100_mae')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm100_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm100_mae')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm100_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm100_mae')

In [129]:
array2tiff(ndvi_ea_simplex_bo_pi18_ed6_anom_sm200_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_bo_pi18_ed6_anom_sm200_mae')
array2tiff(ndvi_ea_simplex_sh_pi18_ed6_anom_sm200_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_sh_pi18_ed6_anom_sm200_mae')
array2tiff(ndvi_ea_simplex_gr_pi18_ed6_anom_sm200_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_gr_pi18_ed6_anom_sm200_mae')
array2tiff(ndvi_ea_simplex_ci_pi18_ed6_anom_sm200_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_ci_pi18_ed6_anom_sm200_mae')
array2tiff(ndvi_ea_simplex_cr_pi18_ed6_anom_sm200_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_cr_pi18_ed6_anom_sm200_mae')

In [32]:
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm10et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm10et_rho')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm40et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm40et_rho')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm100et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm100et_rho')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm200et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm200et_rho')

In [37]:
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm10_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm10_rho')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm40_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm40_rho')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm100_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm100_rho')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm200_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm200_rho')

array2tiff(ndvi_ea_simplex_pi18_ed6_anom_et_rho, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_et_rho')


In [38]:
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm10_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm10_mae')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm40_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm40_mae')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm100_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm100_mae')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm200_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm200_mae')

array2tiff(ndvi_ea_simplex_pi18_ed6_anom_et_mae, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_et_mae')


In [39]:
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm10_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm10_nse')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm40_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm40_nse')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm100_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm100_nse')
array2tiff(ndvi_ea_simplex_pi18_ed6_anom_sm200_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_sm200_nse')

array2tiff(ndvi_ea_simplex_pi18_ed6_anom_et_nse, ndvi_ea, EA_prediction_skill, 'ndvi_ea_simplex_pi18_ed6_anom_et_nse')


# Clip Rasters to EA boundary

In [479]:
def mask_raster_shp(shp, raster, out_name, in_dir, out_dir):
    
    '''
    This function masks a raster to the shape of a smaller shapefile and creates a new raster
    
    '''
    with fiona.open(shp, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]
    with rasterio.open(in_dir + raster) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, nodata = 0, crop=True)
        out_meta = src.meta

    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(out_dir + out_name, "w", **out_meta) as dest:
          
        dest.write(out_image)

In [ ]:
#mask_raster_shp(ea_adminbds, 'ndvi_ea_simplex_uni.tif', 'ndvi_eastafrica_simplex_uni.tif', Africa_simplex, Africa_simplex)

#mask_raster_shp(ea_adminbds, 'ndvi_ea_simplex_multi.tif', 'ndvi_eastafrica_simplex_multi.tif', Africa_simplex, Africa_simplex)